This notebook will be used to scrape data from https://www.musixmatch.com/ and save it in a .csv file.

In [ ]:
import urllib
from urllib import request
import random
import time
import re
import pandas as pd

In [ ]:
#sometimes when the website doesn't have the lyrics for a song, the lyrics on the page will be shown as the following:
no_lyrics = 'Hang on. We took anchor like a ship from distant times. We all started a wonderful journey together. Warm hearts, saints, truths, beautiful smiles. How nice we loved each other. Time has changed like a dark shadow and iceberg. This long journey was very tiring. The expected major collision is taking place. Cold hearts, laws, lies, false smiles. Everybody you love is cruelly ignoring. The sun no longer rises in the morning. The last part of the game will be very dramatic. We forgot Noahs breath. Why did everyone become enemies? Living is not a game. Living is not a game. This is not a game. Hang on a little longer. Hang on. Hang on. Hang on. Hang on a little longer. Hang on. Hang on. Hang on. I feel like shes gonna fall. My journey is over. Stop the world. Take me down. The dream is over. The dream is over. The dream is over. He said wake up. Wake up. Hang on a little longer. Hang on. Hang on. Hang on. Hang on a little longer. Hang on. Hang on. Hang on.'

In [ ]:
rap_album_urls = [] #temporarily empty for display purposes, actual urls used are listed at the bottom of this notebook
rb_album_urls = []

rap_urls = []
rap_song_names = []
rb_urls = []
rb_song_names = []

def get_links_from_album(album_urls, song_names):
  song_urls = []
  i = 1
  for url in album_urls:
    time.sleep(10)
    request = urllib.request.Request(url,headers={'User-Agent': user_agent})
    try:
      html = urllib.request.urlopen(request).read()
      match_urls = re.findall('\"><a href=\"(\/lyrics/.*?)\"', str(html)) #likes
      match_titles = re.findall('\"xboxmusicId\":.*?\"name\":\"(.*?)\",\"nameTranslationList\":', str(html))
      for song_url in match_urls:
        song_urls.append("https://www.musixmatch.com" + song_url)
      for title in match_titles:
        song_names.append(str(title))
      #print(i)
    except:
      print('failure')
      continue
    i += 1
  print(song_urls)
  return song_urls

rap_urls = get_links_from_album(rap_album_urls, rap_song_names)
print("moving on to r&b")
rb_urls = get_links_from_album(rb_album_urls, rb_song_names)

In [ ]:
#make sure duplicate song names are more easily detected
rap_song_names = list(map(lambda x:x.lower(), rap_song_names))
rb_song_names = list(map(lambda x:x.lower(), rb_song_names))
country_song_names = list(map(lambda x:x.lower(), country_song_names))

rap_song_names = list(map(lambda x: re.sub(' \(.*?\)', '', x), rap_song_names))
rb_song_names = list(map(lambda x: re.sub(' \(.*?\)', '', x), rb_song_names))
country_song_names = list(map(lambda x: re.sub(' \(.*?\)', '', x), country_song_names))

In [ ]:
#get lyrics from song urls
lyrics_list = []
titles_list = []
genres = []

def get_lyrics(urls, titles, genre):
  i = 1
  for url in urls:
    if i > 1 and titles[i-1] == titles[i-2]:
        print(str(i) + ': duplicate')
        i += 1
        continue
    time.sleep(random.randint(5,15))
    request = urllib.request.Request(url,headers={'User-Agent': user_agent})
    try:
      html = urllib.request.urlopen(request).read()
      match_lyrics = re.search('\"lyrics\":.*?body\":\"(.*?)\",', str(html)) #likes
      lyrics = match_lyrics.group(1)
      lyrics = re.sub(r"\\n", ' ', lyrics)
      lyrics = re.sub(r"\\+", '', lyrics)
      lyrics_final = re.sub(r"'", "", str(lyrics))
      print(i)
      if lyrics_final == no_lyrics:
        i += 1
        continue
      lyrics_list.append(lyrics_final)
      titles_list.append(titles[i-1])
      genres.append(genre)
    except:
      print('failure')
    i += 1
    if i % 75 == 0 and i != 0:
      print('taking a 15 min nap')
      time.sleep(random.randint(840,960))

get_lyrics(rap_urls, rap_song_names, "Rap")
get_lyrics(rb_urls, rb_song_names, "R&B")

In [ ]:
data = {'Title' : titles_list, 'Lyrics' : lyrics_list, 'Genre' : genres}
df = pd.DataFrame(data)

In [ ]:
#one final step to remove any more duplicates
df.Title = df.Title.replace(to_replace =' \(.*?\)', value = '', regex = True).str.lower()
df = df.drop_duplicates(subset=['Title'], ignore_index=True)

In [ ]:
df.to_csv('rap_r&b13.csv')  #save table

Below are all of the album urls I used:

In [ ]:
#get song urls from albums
#rap_album_urls = ['https://www.musixmatch.com/album/Kid-Cudi/Man-On-the-Moon-III-The-Chosen', 'https://www.musixmatch.com/album/Roddy-Ricch/Please-Excuse-Me-For-Being-Antisocial', 'https://www.musixmatch.com/album/Travis-Scott/ASTROWORLD', 'https://www.musixmatch.com/album/Lil-Uzi-Vert/Eternal-Atake', 'https://www.musixmatch.com/album/Lil-Yachty/Lil-Boat-3-1', 'https://www.musixmatch.com/album/21-Savage/SAVAGE-MODE-II', 'https://www.musixmatch.com/album/Lil-Tecca/Virgo-World', 'https://www.musixmatch.com/album/G-Herbo/PTSD', 'https://www.musixmatch.com/album/Polo-G-2/THE-GOAT', 'https://www.musixmatch.com/album/Gunna-3/WUNNA-2']
#rap_album_urls = ['https://www.musixmatch.com/album/Juice-Wrld/Legends-Never-Die', 'https://www.musixmatch.com/album/Pop-Smoke/Shoot-For-The-Stars-Aim-For-The-Moon', 'https://www.musixmatch.com/album/Juice-Wrld/Legends-Never-Die', 'https://www.musixmatch.com/album/Megan-Thee-Stallion/Good-News-3', 'https://www.musixmatch.com/album/Lil-Baby-4/My-Turn', 'https://www.musixmatch.com/album/The-Kid-Laroi/F-CK-LOVE-1', 'https://www.musixmatch.com/album/Juice-Wrld/Death-Race-for-Love-2', 'https://www.musixmatch.com/album/Juice-Wrld/Goodbye-Good-Riddance', 'https://www.musixmatch.com/album/Eminem/Music-To-Be-Murdered-By-Side-B-Deluxe-Edition', 'https://www.musixmatch.com/album/Eminem/Music-To-Be-Murdered-By-1']
#rap_album_urls = ['https://www.musixmatch.com/album/Big-Sean/Detroit-2', 'https://www.musixmatch.com/album/Logic-1/No-Pressure', 'https://www.musixmatch.com/album/Cardi-B/Invasion-of-Privacy-2', 'https://www.musixmatch.com/album/Smokepurpp-2/Deadstar-2', 'https://www.musixmatch.com/album/Smokepurpp-2/Florida-Jit', 'https://www.musixmatch.com/album/Denzel-Curry/ZUU', 'https://www.musixmatch.com/album/Denzel-Curry/TA13OO-3', 'https://www.musixmatch.com/album/Nav-6/Emergency-Tsunami', 'https://www.musixmatch.com/album/Nav-6/Good-Intentions', 'https://www.musixmatch.com/album/Nav-6/Bad-Habits']
#rap_album_urls = ['https://www.musixmatch.com/album/DaBaby/My-Brother-s-Keeper-Long-Live-G', 'https://www.musixmatch.com/album/DaBaby/BLAME-IT-ON-BABY', 'https://www.musixmatch.com/album/DaBaby/KIRK-2', 'https://www.musixmatch.com/album/DaBaby/Baby-On-Baby', 'https://www.musixmatch.com/album/Pusha-T/DAYTONA', 'https://www.musixmatch.com/album/Meek-Mill/QUARANTINE-PACK', 'https://www.musixmatch.com/album/Sheck-Wes/MUDBOY', 'https://www.musixmatch.com/album/6IX9INE/TattleTales', 'https://www.musixmatch.com/album/6IX9INE/DUMMY-BOY', 'https://www.musixmatch.com/album/Migos/Culture-II']
#rb_album_urls = ['https://www.musixmatch.com/album/The-Weeknd-3/After-Hours-1', 'https://www.musixmatch.com/album/The-Weeknd/Beauty-Behind-the-Madness', 'https://www.musixmatch.com/album/SZA/Ctrl', 'https://www.musixmatch.com/album/Jhene-Aiko/Chilombo', 'https://www.musixmatch.com/album/Khali-D/Free-Spirit-2', 'https://www.musixmatch.com/album/Khali-D/Suncity', 'https://www.musixmatch.com/album/Khali-D/American-Teen', 'https://www.musixmatch.com/album/Doja-Cat/Hot-Pink', 'https://www.musixmatch.com/album/Summer-Walker/Over-It-1', 'https://www.musixmatch.com/album/Ty-Dolla-ign-2/uring-Ty-Dolla-ign-1']
#rb_album_urls = ['https://www.musixmatch.com/album/Kehlani-2/It-Was-Good-Until-It-Wasn-t', 'https://www.musixmatch.com/album/Chloe-x-Halle/Ungodly-Hour-1', 'https://www.musixmatch.com/album/Brent-Faiyaz/Fuck-The-World', 'https://www.musixmatch.com/album/Teyana-Taylor/The-Album', 'https://www.musixmatch.com/album/Brandy/B7', 'https://www.musixmatch.com/album/Giveon/When-It-s-All-Said-And-Done', 'https://www.musixmatch.com/album/John-Legend/Bigger-Love-4', 'https://www.musixmatch.com/album/Thundercat/It-Is-What-It-Is', 'https://www.musixmatch.com/album/dvsn/Amusing-Her-Feelings', 'https://www.musixmatch.com/album/Trey-Songz/Back-Home']
#rb_album_urls = ['https://www.musixmatch.com/album/K-Michelle/Supahood', 'https://www.musixmatch.com/album/Luke-James/blow', 'https://www.musixmatch.com/album/Durand-Bernarr/Dur', 'https://www.musixmatch.com/album/Alicia-Keys/ALICIA', 'https://www.musixmatch.com/album/Jessie-Reyez/BEFORE-LOVE-CAME-TO-KILL-US-2', 'https://www.musixmatch.com/album/Ledisi/The-Wild-Card', 'https://www.musixmatch.com/album/Giveon/TAKE-TIME', 'https://www.musixmatch.com/album/Aloe-Blacc/All-Love-Everything-1', 'https://www.musixmatch.com/album/dvsn/A-Muse-In-Her-Feelings-1', 'https://www.musixmatch.com/album/Ro-James/MANTIC', 'https://www.musixmatch.com/album/Kehlani-2/While-We-Wait']
#rb_album_urls = ['https://www.musixmatch.com/album/Chris-Brown/Indigo', 'https://www.musixmatch.com/album/Jeremih/MIH-TY', 'https://www.musixmatch.com/album/6LACK-1/6pc-Hot-EP', 'https://www.musixmatch.com/album/6lack/East-Atlanta-Love-Letter-2', 'https://www.musixmatch.com/album/Masego/Lady-Lady-2', 'https://www.musixmatch.com/album/Jacquees-4/King-of-R-B', 'https://www.musixmatch.com/album/Jacquees-4/Christmas-In-Decatur', 'https://www.musixmatch.com/album/Jacquees/4275-3', 'https://www.musixmatch.com/album/Ne-Yo/GOOD-MAN-2', 'https://www.musixmatch.com/album/Bryson-Tiller/A-N-N-I-V-E-R-S-A-R-Y-1', 'https://www.musixmatch.com/album/Ari-Lennox/Shea-Butter-Baby-2', 'https://www.musixmatch.com/album/PARTYNEXTDOOR/PARTYPACK', 'https://www.musixmatch.com/album/PARTYNEXTDOOR/PARTYMOBILE-2']
#rb_album_urls = ['https://www.musixmatch.com/album/Lucky-Daye/Painted', 'https://www.musixmatch.com/album/India-Arie/Worthy-2', 'https://www.musixmatch.com/album/Jamila-Woods/LEGACY-LEGACY', 'https://www.musixmatch.com/album/Solange/When-I-Get-Home', 'https://www.musixmatch.com/album/Anderson-Paak/Ventura', 'https://www.musixmatch.com/album/BJ-the-Chicago-Kid/1123', 'https://www.musixmatch.com/album/Raphael-Saadiq/Jimmy-Lee', 'https://www.musixmatch.com/album/SiR-3/Chasing-Summer', 'https://www.musixmatch.com/album/Gallant/Sweet-Insomnia', 'https://www.musixmatch.com/album/Mahalia/LOVE-AND-COMPROMISE-2', 'https://www.musixmatch.com/album/Johnta-Austin/Love-Sex-Religion', 'https://www.musixmatch.com/album/PJ-Morton/PAUL']
#rap_album_urls = ['https://www.musixmatch.com/album/Freddie-Gibbs/Freddie', 'https://www.musixmatch.com/album/Megan-Thee-Stallion/Suga', 'https://www.musixmatch.com/album/Megan-Thee-Stallion/Fever', 'https://www.musixmatch.com/album/BENNY-THE-BUTCHER/The-Plugs-I-Met', 'https://www.musixmatch.com/album/YBN-Cordae/The-Lost-Boy-3', 'https://www.musixmatch.com/album/Saba-7/CARE-FOR-ME', 'https://www.musixmatch.com/album/J-Cole/KOD', 'https://www.musixmatch.com/album/JID-2/DiCaprio-2', 'https://www.musixmatch.com/album/Royce-da-5-9/The-Allegory', 'https://www.musixmatch.com/album/Royce-da-5-9/Book-of-Ryan', 'https://www.musixmatch.com/album/Vince-Staples-2/FM', 'https://www.musixmatch.com/album/Metro-Boomin/NOT-ALL-HEROES-WEAR-CAPES', 'https://www.musixmatch.com/album/Playboi-Carti/Whole-Lotta-Red', 'https://www.musixmatch.com/album/Playboi-Carti/Die-Lit', 'https://www.musixmatch.com/album/Jay-Rock/Redemption-2']
#rb_album_urls = ['https://www.musixmatch.com/album/Teyana-Taylor/K-T-S-E', 'https://www.musixmatch.com/album/Cautious-Clay/Blood-Type-2', 'https://www.musixmatch.com/album/Lucky-Daye/I', 'https://www.musixmatch.com/album/SiR-3/November', 'https://www.musixmatch.com/album/Marsha-Ambrosius/Nyla','https://www.musixmatch.com/album/Ella-Mai/Ella-Mai', 'https://www.musixmatch.com/album/Jacob-Banks/Village', 'https://www.musixmatch.com/album/Jorja-Smith/Lost-Found-2', 'https://www.musixmatch.com/album/Kali-Uchis-2/Isolation', 'https://www.musixmatch.com/album/The-Internet/Hive-Mind', 'https://www.musixmatch.com/album/H-E-R-2/I-Used-To-Know-Her', 'https://www.musixmatch.com/album/Vanjess/Silk-Canvas', 'https://www.musixmatch.com/album/Macy-Gray/Ruby-2', 'https://www.musixmatch.com/album/Leon-Bridges/Good-Thing', 'https://www.musixmatch.com/album/Mariah-Carey/Caution-2']
#rb_album_urls = ['https://www.musixmatch.com/album/Jhene-Aiko/Trip', 'https://www.musixmatch.com/album/Mary-J-Blige/Strength-of-a-Woman-2', 'https://www.musixmatch.com/album/Chloe-x-Halle/The-Two-of-Us', 'https://www.musixmatch.com/album/Brent-Faiyaz/Sonder-Son', 'https://www.musixmatch.com/album/H-E-R-2/H-E-R', 'https://www.musixmatch.com/album/Majid-Jordan/The-Space-Between', 'https://www.musixmatch.com/album/Moses-Sumney-2/Aromanticism', 'https://www.musixmatch.com/album/Thundercat/Drunk-6', 'https://www.musixmatch.com/album/Kehlani-2/SweetSexySavage', 'https://www.musixmatch.com/album/Syd-7/Fin', 'https://www.musixmatch.com/album/Sampha/Process', 'https://www.musixmatch.com/album/Daniel-Caesar/Freudian', 'https://www.musixmatch.com/album/Kelela/Take-Me-Apart', 'https://www.musixmatch.com/album/Sharon-Jones-The-Dap-Kings-2/Soul-of-a-Woman']
#rap_album_urls = ['https://www.musixmatch.com/album/Tee-Grizzley/The-Smartest', 'https://www.musixmatch.com/album/Tee-Grizzley/Scriptures', 'https://www.musixmatch.com/album/Tee-Grizzley/Still-My-Moment', 'https://www.musixmatch.com/album/Tee-Grizzley/Activated', 'https://www.musixmatch.com/album/Lil-Pump/Harverd-Dropout-3', 'https://www.musixmatch.com/album/Lil-Pump/Lil-Pump-2', 'https://www.musixmatch.com/album/Lil-Keed-2/Trapped-On-Cleveland-3', 'https://www.musixmatch.com/album/Lil-Keed-2/Long-Live-Mexico', 'https://www.musixmatch.com/album/NLE-Choppa/From-Dark-to-Light', 'https://www.musixmatch.com/album/NLE-Choppa/Top-Shotta', 'https://www.musixmatch.com/album/2-Chainz/Rap-Or-Go-To-The-League', 'https://www.musixmatch.com/album/2-Chainz/Pretty-Girls-Like-Trap-Music-6', 'https://www.musixmatch.com/album/Gucci-Mane-14/So-Icy-Gang-Vol-1', 'https://www.musixmatch.com/album/Gucci-Mane-14/East-Atlanta-Santa-3-1', 'https://www.musixmatch.com/album/Gucci-Mane-11/Woptober-II-1', 'https://www.musixmatch.com/album/Gucci-Mane/Delusions-of-Grandeur', 'https://www.musixmatch.com/album/Gucci-Mane/Evil-Genius-2']
#country_album_urls = ['https://www.musixmatch.com/album/Brett-Kissel/Now-or-Never', 'https://www.musixmatch.com/album/Dustin-Lynch/Good-Girl', 'https://www.musixmatch.com/album/Marcus-King/El-Dorado-1', 'https://www.musixmatch.com/album/Muscadine-Bloodline/Burn-It-at-Both-Ends-1', 'https://www.musixmatch.com/album/Muscadine-Bloodline/Boys', 'https://www.musixmatch.com/album/Brent-Cobb/Keep-Em-on-They-Toes', 'https://www.musixmatch.com/album/Brent-Cobb/Providence-Canyon', 'https://www.musixmatch.com/album/Brent-Cobb/Shine-on-Rainy-Day-2', 'https://www.musixmatch.com/album/The-Cadillac-Three-1/COUNTRY-FUZZ', 'https://www.musixmatch.com/album/The-Cadillac-Three/Legacy', 'https://www.musixmatch.com/album/John-Moreland/East-October', 'https://www.musixmatch.com/album/The-SteelDrivers/Bad-For-You', 'https://www.musixmatch.com/album/Tenille-Townes/The-Lemonade-Stand', 'https://www.musixmatch.com/album/Carly-Pearce/Closer-To-You', 'https://www.musixmatch.com/album/Carly-Pearce/Every-Little-Thing-2', 'https://www.musixmatch.com/album/Stephanie-Quayle/If-I-Was-a-Cowboy-Acoustic', 'https://www.musixmatch.com/album/Jon-Wolfe/Feels-Like-Country-Music', 'https://www.musixmatch.com/album/Jon-Wolfe/Any-Night-in-Texas', 'https://www.musixmatch.com/album/Hailey-Whitters/The-Days-1', 'https://www.musixmatch.com/album/Sarah-Morey-2/A-Long-Way-to-Go-EP', 'https://www.musixmatch.com/album/The-Secret-Sisters/Cabin-Single', 'https://www.musixmatch.com/album/The-Secret-Sisters/You-Don-t-Own-Me-Anymore', 'https://www.musixmatch.com/album/Trea-Landon/Dirt-Road-Dancin']